This notebook demonstrates building optimization formulations of neural networks using the JuMP modelling language.

Each example uses a similar workflow to the Pyomo-based examples:
- Create an `OmltBlockJuMP` as our model.
- Add variables `x` and `y` where we intend to minimize `y`.
- Use the ONNX parser to import the neural network into a OMLT `NetworkDefinition` object.
- Create a formulation object. 
- Build the formulation object on the `OmltBlockJuMP`.
- Add constraints connecting `x` to the neural network input and `y` to the neural network output.
- Solve with an optimization solver (this example uses Ipopt).
- Query the solution.

### Import necessary packages:

In [1]:
using PythonCall
using JuMP
using Ipopt
using HiGHS

omlt = pyimport("omlt")
omlt_julia = pyimport("omlt.base.julia")
omlt_io = pyimport("omlt.io")
omlt_nn = pyimport("omlt.neuralnet")

onnx_py = pyimport("onnx")


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


Python: <module 'onnx' from '/home/codespace/.julia/environments/v1.10/.CondaPkg/env/lib/python3.12/site-packages/onnx/__init__.py'>

## Full Space Smooth formulation
### Create a model and add `x` and `y` variables and an objective.

In [2]:
fs_model = omlt_julia.OmltBlockJuMP()
fs_model.set_optimizer(Ipopt.Optimizer)

jump_model = pyconvert(Model, fs_model.get_model())

@variable(jump_model, x)
@variable(jump_model, y)
@objective(jump_model, Min, y)

y

#### Create a scaler object, import an ONNX model, and create a Full Space Smooth formulation object from the model and scaler.

In [3]:
scale_x = (1, 0.5)
scale_y = (-0.25, 0.125)

scaler = omlt.OffsetScaling(
    offset_inputs=[scale_x[1]],
    factor_inputs=[scale_x[2]],
    offset_outputs=[scale_y[1]],
    factor_outputs=[scale_y[2]]
)
scaled_input_bounds = Dict(0 => (0,5))

path = "/workspaces/OMLT/tests/models/keras_linear_131_sigmoid.onnx"

py_model = onnx_py.load(path)
net = omlt_io.load_onnx_neural_network(py_model, scaler, scaled_input_bounds)
formulation = omlt_nn.FullSpaceSmoothNNFormulation(net)

Python: <omlt.neuralnet.nn_formulation.FullSpaceSmoothNNFormulation object at 0x749ea7111490>

### Build the formulation onto the model.

In [4]:
fs_model.build_formulation(formulation)

Python: None

### Connect the `x` and `y` variables to the inputs and outputs of the neural network.

In [5]:
@constraint(jump_model, x == pyconvert(VariableRef, fs_model._varrefs["inputs_0"]))
@constraint(jump_model, y == pyconvert(VariableRef, fs_model._varrefs["outputs_0"]))

y - outputs_0 = 0

### Solve the model.

In [6]:
optimize!(jump_model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       30
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:       15
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:       14
Total number of inequality c

### Examine the solution.

In [7]:
println("Full Space Solution:")
println("# of variables: ", num_variables(jump_model))
println("# of constraints: ", num_constraints(jump_model, count_variable_in_set_constraints=false))
println("x = ", value(x))
println("y = ", value(y))
println("Solve Time: ", solve_time(jump_model))

Full Space Solution:
# of variables: 15
# of constraints: 14
x = 1.0000000048214603
y = -0.2534599141581973
Solve Time: 0.05983304977416992


## ReLU Partition-Based Formulation

Do the same things, but for a model with ReLU activation.

In [8]:
rp_model = omlt_julia.OmltBlockJuMP()
rp_model.set_optimizer(HiGHS.Optimizer)

jump_model = pyconvert(Model, rp_model.get_model())

@variable(jump_model, x)
@variable(jump_model, y)
@objective(jump_model, Min, y)

scale_x = (1, 0.5)
scale_y = (-0.25, 0.125)

scaler = omlt.OffsetScaling(
    offset_inputs=[scale_x[1]],
    factor_inputs=[scale_x[2]],
    offset_outputs=[scale_y[1]],
    factor_outputs=[scale_y[2]]
)
scaled_input_bounds = Dict(0 => (0,5))

path = "/workspaces/OMLT/tests/models/keras_linear_131_relu.onnx"

py_model = onnx_py.load(path)
net = omlt_io.load_onnx_neural_network(py_model, scaler, scaled_input_bounds)
formulation = omlt_nn.ReluPartitionFormulation(net)

rp_model.build_formulation(formulation)

Python: None

In [9]:
@constraint(jump_model, x == pyconvert(VariableRef, rp_model._varrefs["inputs_0"]))
@constraint(jump_model, y == pyconvert(VariableRef, rp_model._varrefs["outputs_0"]))

optimize!(jump_model)

Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-01, 3e+00]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 5e+00]
  RHS    [2e-01, 3e+00]
Presolving model
18 rows, 10 cols, 45 nonzeros  0s
13 rows, 5 cols, 26 nonzeros  0s
7 rows, 5 cols, 16 nonzeros  0s
5 rows, 3 cols, 12 nonzeros  0s

Solving MIP model with:
   5 rows
   3 cols (1 binary, 0 integer, 0 implied int., 2 continuous)
   12 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -0.2550872506   inf                  inf        0      0      0         0     0.0s
         1       0         1 100.00%   -0.2510967209   -0.2510967209      0.00%        0      0      0         2     0.0s

Solving report
  Status            Optim

In [10]:
println("Partition-based Solution:")
println("# of variables: ", num_variables(jump_model))
println("# of constraints: ", num_constraints(jump_model, count_variable_in_set_constraints=false))
println("x = ", value(x))
println("y = ", value(y))
println("Solve Time: ", solve_time(jump_model))

Partition-based Solution:
# of variables: 21
# of constraints: 29
x = 1.0
y = -0.2510967209242938
Solve Time: 0.0015048980712890625


Additional ways to look at the model and solution:

In [11]:
jump_model

A JuMP Model
├ solver: HiGHS
├ objective_sense: MIN_SENSE
│ └ objective_function_type: VariableRef
├ num_variables: 21
├ num_constraints: 34
│ ├ AffExpr in MOI.EqualTo{Float64}: 11
│ ├ AffExpr in MOI.GreaterThan{Float64}: 9
│ ├ AffExpr in MOI.LessThan{Float64}: 9
│ ├ VariableRef in MOI.GreaterThan{Float64}: 1
│ ├ VariableRef in MOI.LessThan{Float64}: 1
│ └ VariableRef in MOI.ZeroOne: 3
└ Names registered in the model
  └ :x, :y

In [12]:
all_variables(jump_model)

21-element Vector{VariableRef}:
 x
 y
 inputs_0
 outputs_0
 scaled_inputs_0
 scaled_outputs_0
 layer_128224756400048_z_(0,)
 layer_128224754001408_z_(0,)
 layer_128224754001408_z_(1,)
 layer_128224754001408_z_(2,)
 layer_128224754001408_zhat_(0,)
 layer_128224754001408_zhat_(1,)
 layer_128224754001408_zhat_(2,)
 layer_128224756396496_z_(0,)
 layer_128224756396496_zhat_(0,)
 layer_128224754001408_output_node_block_(0,)_sig
 layer_128224754001408_output_node_block_(0,)_z2_0
 layer_128224754001408_output_node_block_(1,)_sig
 layer_128224754001408_output_node_block_(1,)_z2_0
 layer_128224754001408_output_node_block_(2,)_sig
 layer_128224754001408_output_node_block_(2,)_z2_0

In [ ]:
all_constraints(jump_model,include_variable_in_set_constraints=true)

In [ ]:
solution_summary(jump_model)